## **Assignment 5 - Machine Learning - LB01**
### **Fendy Wijaya - 2602092150**
### **Spotify Recommendation System**

In [ ]:
# Import data from kaggle

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'spotify-recommendation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1495083%2F2469778%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240404%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240404T041117Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc00afd2ef2c4e553453f67d49f8d422f76ffac43974ef01d4fed2d46aba02a16d507ec2078c313edaa739a5e424ddb61762a5503f26060706d19b45bfc376f94fa485e17e6ca247ef32a215f54b19c05c848c9936ea2daa310acc3e42534b9571cc43f9c5277110d3d3c1e81d30cd6a7287c10385fef62807d4f18acef725144bb7f6df47d72e4cb353319e1683c519990cdf2686fd81d541472e068753103657a3c30b4f8ae2f8be1504efd5c0e87e24c62c51b264b9fd0d8565b9007bd81e479437df402baf46a7eca037ece33101424e22e9f0799ebae475535fb9109d2530008db7013d26f05d5b5fa253f39e0efd1207c0fc7f5efeb9f366398c8cc5a35'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 30201 bytes downloaded
Downloaded and uncompressed: spotify-recommendation
Data source import complete.


In [ ]:
# Read all data in kaggle/input

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spotify-recommendation/good.json
/kaggle/input/spotify-recommendation/no.py
/kaggle/input/spotify-recommendation/yes.py
/kaggle/input/spotify-recommendation/dislike.json
/kaggle/input/spotify-recommendation/data.csv


In [ ]:
df = pd.read_csv('/kaggle/input/spotify-recommendation/data.csv')
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,liked
0,0.803,0.6240,7,-6.764,0,0.0477,0.4510,0.000734,0.1000,0.6280,95.968,304524,4,0
1,0.762,0.7030,10,-7.951,0,0.3060,0.2060,0.000000,0.0912,0.5190,151.329,247178,4,1
2,0.261,0.0149,1,-27.528,1,0.0419,0.9920,0.897000,0.1020,0.0382,75.296,286987,4,0
3,0.722,0.7360,3,-6.994,0,0.0585,0.4310,0.000001,0.1230,0.5820,89.860,208920,4,1
4,0.787,0.5720,1,-7.516,1,0.2220,0.1450,0.000000,0.0753,0.6470,155.117,179413,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.166,0.0551,9,-19.494,0,0.0520,0.9760,0.635000,0.1190,0.1430,176.616,206520,3,0
191,0.862,0.6240,3,-11.630,1,0.0565,0.0192,0.000153,0.0465,0.8820,124.896,254240,4,0
192,0.499,0.3510,9,-11.509,0,0.0448,0.9510,0.000099,0.1180,0.6160,90.664,235947,4,0
193,0.574,0.7290,10,-5.838,0,0.0965,0.0406,0.000004,0.1940,0.4130,110.547,190239,5,1


In [ ]:
import json

with open('../input/spotify-recommendation/good.json', 'r') as f:
    data = json.load(f)

good_df = pd.DataFrame(data['audio_features'])
good_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.749,0.839,6,-4.847,1,0.2970,0.0867,0.000000,0.2040,0.804,172.068,audio_features,55mcupbf7cIsuCEVAuTJVk,spotify:track:55mcupbf7cIsuCEVAuTJVk,https://api.spotify.com/v1/tracks/55mcupbf7cIs...,https://api.spotify.com/v1/audio-analysis/55mc...,111000,4
1,0.573,0.581,10,-9.026,0,0.3390,0.7530,0.000001,0.1300,0.351,76.506,audio_features,57RtLWT7IpugV0yi5bsxJk,spotify:track:57RtLWT7IpugV0yi5bsxJk,https://api.spotify.com/v1/tracks/57RtLWT7Ipug...,https://api.spotify.com/v1/audio-analysis/57Rt...,169347,4
2,0.800,0.719,7,-6.262,1,0.2340,0.1090,0.000000,0.0580,0.815,143.975,audio_features,5VyfAfp2Yt3qaeuvq55ll3,spotify:track:5VyfAfp2Yt3qaeuvq55ll3,https://api.spotify.com/v1/tracks/5VyfAfp2Yt3q...,https://api.spotify.com/v1/audio-analysis/5Vyf...,230854,4
3,0.778,0.632,8,-6.415,1,0.1250,0.0404,0.000000,0.0912,0.827,140.951,audio_features,3eWHY75nDgte70hh5yf4UW,spotify:track:3eWHY75nDgte70hh5yf4UW,https://api.spotify.com/v1/tracks/3eWHY75nDgte...,https://api.spotify.com/v1/audio-analysis/3eWH...,224029,4
4,0.797,0.852,8,-5.202,1,0.2410,0.0555,0.000024,0.0536,0.480,136.035,audio_features,2UwrB6Ge6mPfUV8yGvAfX7,spotify:track:2UwrB6Ge6mPfUV8yGvAfX7,https://api.spotify.com/v1/tracks/2UwrB6Ge6mPf...,https://api.spotify.com/v1/audio-analysis/2Uwr...,102353,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.616,0.534,10,-10.264,0,0.4830,0.6390,0.000000,0.0844,0.556,170.054,audio_features,3dl4lXWlOxnGo94OqNtpdq,spotify:track:3dl4lXWlOxnGo94OqNtpdq,https://api.spotify.com/v1/tracks/3dl4lXWlOxnG...,https://api.spotify.com/v1/audio-analysis/3dl4...,146480,4
96,0.802,0.549,5,-8.600,0,0.0631,0.2680,0.004960,0.0984,0.498,138.984,audio_features,3DWDcsDoXRIFynCREIkibM,spotify:track:3DWDcsDoXRIFynCREIkibM,https://api.spotify.com/v1/tracks/3DWDcsDoXRIF...,https://api.spotify.com/v1/audio-analysis/3DWD...,184627,4
97,0.767,0.659,7,-4.541,1,0.0387,0.7850,0.000000,0.1120,0.631,90.056,audio_features,6fmtZEUoGwxPNvUgr0NJm1,spotify:track:6fmtZEUoGwxPNvUgr0NJm1,https://api.spotify.com/v1/tracks/6fmtZEUoGwxP...,https://api.spotify.com/v1/audio-analysis/6fmt...,186720,4
98,0.752,0.468,0,-9.966,1,0.3330,0.8050,0.000000,0.1360,0.716,82.795,audio_features,2edc1m7WXxdBA61WlxSInp,spotify:track:2edc1m7WXxdBA61WlxSInp,https://api.spotify.com/v1/tracks/2edc1m7WXxdB...,https://api.spotify.com/v1/audio-analysis/2edc...,179253,4


In [ ]:
good_df = good_df.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])
good_df['like'] = 1
good_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,like
0,0.749,0.839,6,-4.847,1,0.2970,0.0867,0.000000,0.2040,0.804,172.068,55mcupbf7cIsuCEVAuTJVk,4,1
1,0.573,0.581,10,-9.026,0,0.3390,0.7530,0.000001,0.1300,0.351,76.506,57RtLWT7IpugV0yi5bsxJk,4,1
2,0.800,0.719,7,-6.262,1,0.2340,0.1090,0.000000,0.0580,0.815,143.975,5VyfAfp2Yt3qaeuvq55ll3,4,1
3,0.778,0.632,8,-6.415,1,0.1250,0.0404,0.000000,0.0912,0.827,140.951,3eWHY75nDgte70hh5yf4UW,4,1
4,0.797,0.852,8,-5.202,1,0.2410,0.0555,0.000024,0.0536,0.480,136.035,2UwrB6Ge6mPfUV8yGvAfX7,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.616,0.534,10,-10.264,0,0.4830,0.6390,0.000000,0.0844,0.556,170.054,3dl4lXWlOxnGo94OqNtpdq,4,1
96,0.802,0.549,5,-8.600,0,0.0631,0.2680,0.004960,0.0984,0.498,138.984,3DWDcsDoXRIFynCREIkibM,4,1
97,0.767,0.659,7,-4.541,1,0.0387,0.7850,0.000000,0.1120,0.631,90.056,6fmtZEUoGwxPNvUgr0NJm1,4,1
98,0.752,0.468,0,-9.966,1,0.3330,0.8050,0.000000,0.1360,0.716,82.795,2edc1m7WXxdBA61WlxSInp,4,1


In [ ]:
import json

with open('../input/spotify-recommendation/dislike.json', 'r') as f:
    data = json.load(f)

bad_df = pd.DataFrame(data['audio_features'])
bad_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.357,0.980,6,-6.835,1,0.0790,0.000052,0.843000,0.0768,0.368,96.969,audio_features,4pFC6tuWErxbO61oFFq3BQ,spotify:track:4pFC6tuWErxbO61oFFq3BQ,https://api.spotify.com/v1/tracks/4pFC6tuWErxb...,https://api.spotify.com/v1/audio-analysis/4pFC...,242760,4
1,0.446,0.977,10,-5.036,0,0.0781,0.000535,0.472000,0.1050,0.339,172.059,audio_features,6V3SNkvi4BnfmZU0j7s9TQ,spotify:track:6V3SNkvi4BnfmZU0j7s9TQ,https://api.spotify.com/v1/tracks/6V3SNkvi4Bnf...,https://api.spotify.com/v1/audio-analysis/6V3S...,284400,4
2,0.291,0.980,1,-5.138,1,0.1530,0.001270,0.091000,0.1020,0.257,79.792,audio_features,1Ds58cpyJzL0YTKk02vh1C,spotify:track:1Ds58cpyJzL0YTKk02vh1C,https://api.spotify.com/v1/tracks/1Ds58cpyJzL0...,https://api.spotify.com/v1/audio-analysis/1Ds5...,270920,4
3,0.462,0.974,1,-5.820,1,0.0816,0.000029,0.723000,0.0751,0.399,107.877,audio_features,77QKNMGwlnzxsTZPYfByMu,spotify:track:77QKNMGwlnzxsTZPYfByMu,https://api.spotify.com/v1/tracks/77QKNMGwlnzx...,https://api.spotify.com/v1/audio-analysis/77QK...,186576,3
4,0.338,0.988,8,-7.290,0,0.0865,0.000083,0.833000,0.0377,0.449,99.046,audio_features,3bQoMbPYOvvseHaSOWAQ3Q,spotify:track:3bQoMbPYOvvseHaSOWAQ3Q,https://api.spotify.com/v1/tracks/3bQoMbPYOvvs...,https://api.spotify.com/v1/audio-analysis/3bQo...,221960,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.798,0.746,10,-8.639,1,0.0313,0.030400,0.361000,0.0703,0.965,128.553,audio_features,1QqsAlhs7pxcXs9BMB1Nv5,spotify:track:1QqsAlhs7pxcXs9BMB1Nv5,https://api.spotify.com/v1/tracks/1QqsAlhs7pxc...,https://api.spotify.com/v1/audio-analysis/1Qqs...,655213,4
91,0.651,0.811,10,-13.870,1,0.0318,0.064800,0.029300,0.1000,0.962,112.126,audio_features,6VuIKS2Z27jCutIapgCUZF,spotify:track:6VuIKS2Z27jCutIapgCUZF,https://api.spotify.com/v1/tracks/6VuIKS2Z27jC...,https://api.spotify.com/v1/audio-analysis/6VuI...,186573,4
92,0.705,0.847,6,-2.588,1,0.0361,0.414000,0.000003,0.4060,0.743,120.527,audio_features,7sMGwiS4vOMcz86ZY3vKYM,spotify:track:7sMGwiS4vOMcz86ZY3vKYM,https://api.spotify.com/v1/tracks/7sMGwiS4vOMc...,https://api.spotify.com/v1/audio-analysis/7sMG...,228120,4
93,0.819,0.811,6,-7.760,0,0.0349,0.203000,0.005580,0.1390,0.766,106.684,audio_features,4JsQkSOE4taeyhH3pv1een,spotify:track:4JsQkSOE4taeyhH3pv1een,https://api.spotify.com/v1/tracks/4JsQkSOE4tae...,https://api.spotify.com/v1/audio-analysis/4JsQ...,246760,4


In [ ]:
bad_df = bad_df.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])
bad_df['like'] = 0
bad_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,like
0,0.357,0.980,6,-6.835,1,0.0790,0.000052,0.843000,0.0768,0.368,96.969,4pFC6tuWErxbO61oFFq3BQ,4,0
1,0.446,0.977,10,-5.036,0,0.0781,0.000535,0.472000,0.1050,0.339,172.059,6V3SNkvi4BnfmZU0j7s9TQ,4,0
2,0.291,0.980,1,-5.138,1,0.1530,0.001270,0.091000,0.1020,0.257,79.792,1Ds58cpyJzL0YTKk02vh1C,4,0
3,0.462,0.974,1,-5.820,1,0.0816,0.000029,0.723000,0.0751,0.399,107.877,77QKNMGwlnzxsTZPYfByMu,3,0
4,0.338,0.988,8,-7.290,0,0.0865,0.000083,0.833000,0.0377,0.449,99.046,3bQoMbPYOvvseHaSOWAQ3Q,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.798,0.746,10,-8.639,1,0.0313,0.030400,0.361000,0.0703,0.965,128.553,1QqsAlhs7pxcXs9BMB1Nv5,4,0
91,0.651,0.811,10,-13.870,1,0.0318,0.064800,0.029300,0.1000,0.962,112.126,6VuIKS2Z27jCutIapgCUZF,4,0
92,0.705,0.847,6,-2.588,1,0.0361,0.414000,0.000003,0.4060,0.743,120.527,7sMGwiS4vOMcz86ZY3vKYM,4,0
93,0.819,0.811,6,-7.760,0,0.0349,0.203000,0.005580,0.1390,0.766,106.684,4JsQkSOE4taeyhH3pv1een,4,0


In [ ]:
df = good_df.merge(bad_df, how='outer')
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,like
0,0.749,0.839,6,-4.847,1,0.2970,0.0867,0.000000,0.2040,0.804,172.068,55mcupbf7cIsuCEVAuTJVk,4,1
1,0.573,0.581,10,-9.026,0,0.3390,0.7530,0.000001,0.1300,0.351,76.506,57RtLWT7IpugV0yi5bsxJk,4,1
2,0.800,0.719,7,-6.262,1,0.2340,0.1090,0.000000,0.0580,0.815,143.975,5VyfAfp2Yt3qaeuvq55ll3,4,1
3,0.778,0.632,8,-6.415,1,0.1250,0.0404,0.000000,0.0912,0.827,140.951,3eWHY75nDgte70hh5yf4UW,4,1
4,0.797,0.852,8,-5.202,1,0.2410,0.0555,0.000024,0.0536,0.480,136.035,2UwrB6Ge6mPfUV8yGvAfX7,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.798,0.746,10,-8.639,1,0.0313,0.0304,0.361000,0.0703,0.965,128.553,1QqsAlhs7pxcXs9BMB1Nv5,4,0
191,0.651,0.811,10,-13.870,1,0.0318,0.0648,0.029300,0.1000,0.962,112.126,6VuIKS2Z27jCutIapgCUZF,4,0
192,0.705,0.847,6,-2.588,1,0.0361,0.4140,0.000003,0.4060,0.743,120.527,7sMGwiS4vOMcz86ZY3vKYM,4,0
193,0.819,0.811,6,-7.760,0,0.0349,0.2030,0.005580,0.1390,0.766,106.684,4JsQkSOE4taeyhH3pv1een,4,0


In [ ]:
temp_df = df.drop(columns=['id', 'like'])
temp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.749,0.839,6,-4.847,1,0.2970,0.0867,0.000000,0.2040,0.804,172.068,4
1,0.573,0.581,10,-9.026,0,0.3390,0.7530,0.000001,0.1300,0.351,76.506,4
2,0.800,0.719,7,-6.262,1,0.2340,0.1090,0.000000,0.0580,0.815,143.975,4
3,0.778,0.632,8,-6.415,1,0.1250,0.0404,0.000000,0.0912,0.827,140.951,4
4,0.797,0.852,8,-5.202,1,0.2410,0.0555,0.000024,0.0536,0.480,136.035,4
...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.798,0.746,10,-8.639,1,0.0313,0.0304,0.361000,0.0703,0.965,128.553,4
191,0.651,0.811,10,-13.870,1,0.0318,0.0648,0.029300,0.1000,0.962,112.126,4
192,0.705,0.847,6,-2.588,1,0.0361,0.4140,0.000003,0.4060,0.743,120.527,4
193,0.819,0.811,6,-7.760,0,0.0349,0.2030,0.005580,0.1390,0.766,106.684,4


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(temp_df)
print(cs)

[[1.         0.99116164 0.99977722 ... 0.99980266 0.99866096 0.99803467]
 [0.99116164 1.         0.99370509 ... 0.9920779  0.99620468 0.99594091]
 [0.99977722 0.99370509 1.         ... 0.99973161 0.99947031 0.99893282]
 ...
 [0.99980266 0.9920779  0.99973161 ... 1.         0.99862962 0.99774453]
 [0.99866096 0.99620468 0.99947031 ... 0.99862962 1.         0.99982514]
 [0.99803467 0.99594091 0.99893282 ... 0.99774453 0.99982514 1.        ]]


In [ ]:
songs_ids = pd.Series(df.index, index=df['id'])
songs_ids

id
55mcupbf7cIsuCEVAuTJVk      0
57RtLWT7IpugV0yi5bsxJk      1
5VyfAfp2Yt3qaeuvq55ll3      2
3eWHY75nDgte70hh5yf4UW      3
2UwrB6Ge6mPfUV8yGvAfX7      4
                         ... 
1QqsAlhs7pxcXs9BMB1Nv5    190
6VuIKS2Z27jCutIapgCUZF    191
7sMGwiS4vOMcz86ZY3vKYM    192
4JsQkSOE4taeyhH3pv1een    193
3P2zU7fEjnZRunH124QnJL    194
Length: 195, dtype: int64

In [ ]:
def song_recommender(song_id, cs=cs):
    # Index of song using id
    idx = songs_ids[song_id]
    # Get cosine similarity scores
    scores = list(enumerate(cs[idx]))
    sorted_scores = sorted(scores, key=lambda x:x[1], reverse=True)[1:11]
    # Get index of top 10 similar songs
    rec_songs_idxs = [i[0] for i in sorted_scores]
    print("Is this song good or bad? (good=1, bad=0):", df[df['id']==song_id]['like'].values[0])
    print("\nTop 10 Similar Songs:\n")
    return df[['id']].iloc[rec_songs_idxs]

In [ ]:
candid_id = '55mcupbf7cIsuCEVAuTJVk'
song_recommender(song_id=candid_id)

Is this song good or bad? (good=1, bad=0): 1

Top 10 Similar Songs:



,id
71,03JjVo4OYciAi88SEdtDii
48,6mkAhxAj1y5zvXWefkd9y7
89,0PieJw6d3jyki3JTxgZhIr
36,35tyQCmxn6cj2r7eAb4JXP
88,4QMaRC93F0aFx3S648CLyh
92,0i8JFpqe9cKwnrcvoNgl1L
21,6mKLhdZIMaHpZ9kbmq9SOS
54,205HNJ73cgpC0LAOnuQiWT
68,2fl01uGwbppFEci7qij1lU
44,4VdQCcXDuzb6Ks1yLT9FnO
